# Haydi - How to create your own program

This text presents an introductory guide that should help people to create their own program using **Haydi** framework. Haydi is a framework focusing on combinatorics problems. It allows users to formulate their problems easily and lets them check whether or not certain criteria are met. Moreover, instances confirming or contradicting the hypotheses are provided. The advantage of using Haydi is that the **resulting program can run directly on a supercomputer**, hence quite large structures can be examined. On the other hand, such infrastructure is not demanded and programs may run on one standard PC or distributively among a specific number of PCs.


Here a user can find a **step-by-step tutorial built upon a specific problem**; particulary, Cerny's hypothesis about [synchronizing words](https://en.wikipedia.org/wiki/Synchronizing_word) in deterministic finite automata (DFA). The hypotesis states the question: _if a DFA has a synchronizing word, must it have one of length at most $(n − 1)^2$?_ This is an open question and the resulting program does not have an ambition to solve it. But it shows how much or perhaps how little effort it may take to create such a program and how large instances can be checked with it.

## Definitions

**Deterministic finite automaton**, is a 5-tuple, $(Q,\Sigma, \delta, q_0, F)$, consisting of:
 * $Q$, a finite set of states
 * $\Sigma$, a finite set of symbols called aplphabet
 * $\delta: Q \times S \to Q$, a transition function
 * $q_0 \in Q$, an initial state,
 * $F \subseteq Q$, a subset of final states
 

**Synchronizing word**, is a finite sequnce of symbols from alphabet, $\sigma \in \Sigma^*$, with a property that no matter at which state an automaton starts, following $\sigma$ word leads to the same state, i.e. $\forall q \in Q, \exists q' \in Q: q \xrightarrow{\sigma} q'$.

## Making a program
Here we start making our first program using Haydi framwork. The problem of *synchronizing words* is well-known among people from the area of theoretical computer science. This problem was chosen because it is easily explainable to people while containing enough complexity; such programm will contain most of the structures and constructs that can help you to create your own program.

### Before we start
Before we can even start we need to **install Haydi**. Haydi is available on [GitHub](https://github.com/spirali/haydi) as an open-source project. It is distributed as a module that can be used in [Python](https://www.python.org/) language. All the necessary information about instalaction and project's structure can be found on its page.

### Link the project and import Haidy modul
First of all, when we start making our new program, we need to tell where Haydi modul is located and put the location into program's path. We can do this as follows:

In [6]:
import sys
sys.path.insert(0, "../src") # the program is placed in subdirectory of project's root

At this point we can import Haydi modul. It cointain all of the structures that are par of public API. All the structures with their description can be found in reference manual **#TODO: odkaz**

In [2]:
import haydi as hd

### Prepare the structures
Now we are in state when we succesfully installed Haydi, linked it to our program, and import the module. We can start to make our program.

As stated in the introduction, Haydi is a suitable framework for programs that, either systematicaly or randomly explore a bunch of problem instances and find those satisfying certain criteria. In this context, **the first task** we are going to address, is to define that _bunch_ of problem's instances?

The basic structure in Haydi is a **Domain**. ..

In [3]:
x = hd.Range(4)

In [4]:
print x

<Range size=4 {0, 1, 2, 3}>
